# FizzBuzz
- 从1开始往上数数，当遇到3的倍数时，说fizz
- 当遇到5的倍数说buzz
- 当遇到15的倍数fizzbuzz 
- 其他情况正常数数

In [13]:
def fizz_buzz_encode(i):
    if i % 15==0: return 3
    elif i % 5==0: return 2
    elif i % 3==0: return 1
    else: return 0
    
def fizz_buzz_decode(i,prediction):
    return [str(i),'fizz','buzz','fizzbuzz'][prediction]

In [2]:
def helper(i):
    print(fizz_buzz_decode(i,fizz_buzz_encode(i)))

In [3]:
for i in range(10):
    helper(i)

fizzbuzz
1
2
fizz
4
buzz
fizz
7
8
fizz


首先定义模型的输入和输出(训练数据)

In [18]:
import numpy as np
import torch

NUM_DIGITS = 10 #十位数有1024个数可以预测，100以下做测试数据

#将数字转成二进制数据，比较容易学习
def binary_encode(i,num_digits):
    # 等价于i除以2的d次方，然后与1做与运算，也就是如果i/(2^d)得到的数装换为2进制后，如果最后一位为1，就输出1，最后一位为0就输出0。
    return np.array([i >> d & 1 for d in range(num_digits)])[::-1]

trX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(101,2**NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101,2**NUM_DIGITS)])

binary_encode(16,NUM_DIGITS)

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [19]:
trX

tensor([[0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 1.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [20]:
trY.shape

torch.Size([923])

用pytorch定义模型

In [21]:
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS,NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN,4) # 四分类概率分布
)
if torch.cuda.is_available():
    model=model.cuda()

In [33]:
loss_fn = torch.nn.CrossEntropyLoss() #拟合两种分布相似度有多高
optimizer = torch.optim.Adam(model.parameters(),lr=0.05)

In [34]:
BATCH_SIZE=128

for epoch in range(10000):
    for start in range(0,len(trX),BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]
        
        if torch.cuda.is_available():
            batchX = batchX.cuda()
            batchY = batchY.cuda()
            
        y_pred = model(batchX) #forward
        loss = loss_fn(y_pred,batchY)
        
#         print("Epoch",epoch,loss.item())
        optimizer.zero_grad()
        loss.backward() #backpass
        optimizer.step() #gradient descent

训练好的模型尝试在1到100这些数字上玩

In [35]:
testX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(1,101)])
if torch.cuda.is_available():
    testX = testX.cuda()

with torch.no_grad():
    testY = model(testX)
    


In [36]:
testY.max(1)[1].cpu().data.numpy()

array([0, 1, 1, 0, 0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 2, 0, 0,
       0, 0, 2, 0, 1, 0, 0, 3, 0, 1, 1, 1, 2, 1, 0, 1, 1, 2, 0, 1, 0, 0,
       3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 2, 0, 1, 0, 0, 3, 1, 0, 1, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0,
       0, 3, 0, 0, 0, 0, 2, 1, 0, 1, 1, 1])

In [37]:
predicts = zip(range(1,100),list(testY.max(1)[1].cpu().data.tolist()))

In [38]:
print([fizz_buzz_decode(i,x) for i,x in predicts])

['1', 'fizz', 'fizz', '4', '5', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', '21', '22', '23', '24', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', 'fizz', 'fizz', 'fizz', 'buzz', 'fizz', '37', 'fizz', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', '65', 'fizz', '67', '68', '69', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', 'fizz', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', '85', 'buzz', 'fizz', '88', '89', 'fizzbuzz', '91', '92', '93', '94', 'buzz', 'fizz', '97', 'fizz', 'fizz']
